<a href="https://colab.research.google.com/github/prithwis/AGI/blob/main/SnakeFrog_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)<br>


![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>




[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Primitive Snake-Frog World
for simulating Reinforcement Learning Applications

In [1]:
# 1. Update package list and install the NEW opengl names
!apt-get update > /dev/null 2>&1
!apt-get install -y xvfb ffmpeg freeglut3-dev python3-opengl libgl1-mesa-dev libglu1-mesa-dev mesa-utils > /dev/null 2>&1

# 2. Install the system dependencies (SWIG is the key here)
#!apt-get update
!apt-get install -y swig build-essential python3-dev > /dev/null 2>&1

# 3. Upgrade pip and setuptools to handle the build process better
!pip install --upgrade pip setuptools wheel > /dev/null 2>&1

# 4. Now install gymnasium with box2d support
!pip install "gymnasium[box2d]" > /dev/null 2>&1

# 5. Install the Python libraries
!pip install pyvirtualdisplay  pygame opencv-python > /dev/null 2>&1


In [2]:
import gymnasium as gym
try:
    env = gym.make("LunarLander-v3")
    print("Success! The physics engine is ready for AGI training.")
    env.close()
except Exception as e:
    print(f"Error: {e}")

/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Success! The physics engine is ready for AGI training.


In [3]:
import pygame
import cv2
import numpy as np
import os

from IPython.display import Video, display


In [4]:
# Here the frog moving in a spiral is chased by three snakes and a video is created
#
# --- 1. SETUP ---
WIDTH, HEIGHT = 640, 480
FPS = 30
SECONDS = 10  # Longer simulation to see the spiral grow
raw_video = 'temp_sim.avi'
final_video = 'spiral_chase.mp4'

def create_spiral_chase():
    if os.path.exists(raw_video): os.remove(raw_video)

    pygame.init()
    screen = pygame.Surface((WIDTH, HEIGHT))

    # We use XVID for the initial stable write
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video = cv2.VideoWriter(raw_video, fourcc, FPS, (WIDTH, HEIGHT))

    # Positions
    frog_pos = np.array([300.0, 200.0])
    snakes = [np.array([float(np.random.randint(0, WIDTH)),
                        float(np.random.randint(0, HEIGHT))]) for _ in range(4)]

    print("Simulating Spiral Chase... capturing frames.")

    for t in range(FPS * SECONDS):
        # A. SPIRAL LOGIC (The Frog)
        # Radius increases over time (t * 0.5), creating a spiral
        radius = t * 0.8
        angle = t * 0.15
        frog_pos[0] = 320 + radius * np.cos(angle)
        frog_pos[1] = 240 + radius * np.sin(angle)

        # B. CHASE LOGIC (The Snakes)
        for s_pos in snakes:
            direction = frog_pos - s_pos
            dist = np.linalg.norm(direction)
            if dist > 0:
                # Snakes get slightly faster as the game goes on
                speed = 3.0 + (t * 0.005)
                s_pos += (direction / dist) * speed

        # C. DRAWING
        screen.fill((25, 25, 35)) # Deep space blue
        # Frog (Target)
        pygame.draw.circle(screen, (0, 255, 100), (int(frog_pos[0]), int(frog_pos[1])), 14)
        # Snakes (Agents)
        for s_pos in snakes:
            pygame.draw.rect(screen, (255, 80, 80), (int(s_pos[0]-10), int(s_pos[1]-10), 20, 20))

        # D. CAPTURE & WRITE
        view = pygame.surfarray.array3d(screen)
        view = view.transpose([1, 0, 2])
        view = cv2.cvtColor(view, cv2.COLOR_RGB2BGR)
        video.write(view)

    video.release()
    pygame.quit()



In [5]:
# --- 2. RUN AND CONVERT ---
create_spiral_chase()
print("\n--- CHASE COMPLETE ---")

Simulating Spiral Chase... capturing frames.

--- CHASE COMPLETE ---


In [6]:
# Convert to H.264 MP4 so the browser can play it
!ffmpeg -y -i {raw_video} -c:v libx264 -pix_fmt yuv420p {final_video} -hide_banner -loglevel error


display(Video(final_video, embed=True, width=600))

In [7]:
# --- 1. THE RECORDER CLASS ---
class VideoRecorder:
    def __init__(self, filename='simulation.avi', width=640, height=480, fps=30):
        self.filename = filename
        self.width = width
        self.height = height
        self.fps = fps
        self.fourcc = cv2.VideoWriter_fourcc(*'XVID')
        self.video_writer = None

    def start(self):
        if os.path.exists(self.filename): os.remove(self.filename)
        self.video_writer = cv2.VideoWriter(self.filename, self.fourcc, self.fps, (self.width, self.height))

    def record_frame(self, surface):
        # Convert Pygame surface to OpenCV format
        view = pygame.surfarray.array3d(surface)
        view = view.transpose([1, 0, 2])
        view = cv2.cvtColor(view, cv2.COLOR_RGB2BGR)
        view = cv2.resize(view, (self.width, self.height))
        self.video_writer.write(view)

    def stop(self):
        self.video_writer.release()
        # Convert to MP4 for browser compatibility
        output_mp4 = self.filename.replace('.avi', '.mp4')
        os.system(f"ffmpeg -y -i {self.filename} -c:v libx264 -pix_fmt yuv420p {output_mp4} -hide_banner -loglevel error")
        return output_mp4

In [8]:
# --- 2. THE WORLD CLASS (Logic goes here) ---
class FrogSnakeWorld:
    def __init__(self, width=640, height=480):
        self.width = width
        self.height = height
        self.frog_pos = np.array([width/2, height/2])
        self.frog_vel = np.array([2.0, 2.0])
        self.snakes = [{"pos": np.array([0.0, 0.0]), "speed": 3.0} for _ in range(3)]

    def update(self, step_count):
        # FROG BEHAVIOR: Random break every 15 steps
        if step_count % 15 == 0:
            angle = np.random.uniform(0, 2 * np.pi)
            self.frog_vel = np.array([np.cos(angle), np.sin(angle)]) * 5.0

        self.frog_pos += self.frog_vel

        # Wall Bouncing
        if self.frog_pos[0] <= 0 or self.frog_pos[0] >= self.width: self.frog_vel[0] *= -1
        if self.frog_pos[1] <= 0 or self.frog_pos[1] >= self.height: self.frog_vel[1] *= -1

        # SNAKE BEHAVIOR: Simple Tracking
        for s in self.snakes:
            direction = self.frog_pos - s["pos"]
            dist = np.linalg.norm(direction)
            if dist > 0:
                s["pos"] += (direction / dist) * s["speed"]

    def draw(self, surface):
        surface.fill((25, 25, 30))
        # Draw Frog
        pygame.draw.circle(surface, (0, 255, 150), (int(self.frog_pos[0]), int(self.frog_pos[1])), 12)
        # Draw Snakes
        for s in self.snakes:
            pygame.draw.rect(surface, (255, 80, 80), (int(s["pos"][0]-10), int(s["pos"][1]-10), 20, 20))

In [9]:
# --- 3. THE EXECUTION BRIDGE ---
def run_experiment(seconds=10):
    pygame.init()
    width, height = 640, 480
    surface = pygame.Surface((width, height))

    # Initialize our modular components
    world = FrogSnakeWorld(width, height)
    recorder = VideoRecorder(width=width, height=height)

    recorder.start()
    print("Simulating...")

    for t in range(30 * seconds):
        world.update(t)
        world.draw(surface)
        recorder.record_frame(surface)

    final_file = recorder.stop()
    pygame.quit()

    display(Video(final_file, embed=True, width=600))

In [10]:
run_experiment()

Simulating...


#Chronobooks <br>
Three science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chrono3) <br>
![alt text](https://blogger.googleusercontent.com/img/a/AVvXsEjsZufX_KYaLwAnJP6bUxvDg5RSPn6r8HIZe749nLWX3RuwyshrYEAUpdw03a9WIWRdnzA9epwJOE05eDJ0Ad7kGyfWiUrC2vNuOskb2jA-e8aOZSx8YqzT8mfZi3E4X1Rz3qlEAiv-aTxlCM976BEeTjx4J64ctY3C_FoV4v9aY_U23F8xRqI5Eg=s1600)